<a href="https://colab.research.google.com/github/gp0942/Scryfall2DH/blob/main/Scryfall2DH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# running this cell will load the necessary libraries to run the rest of the workbook

import requests
import json
import pandas as pd
from datetime import date

In [2]:
# set up scryfall api request
response = requests.get('https://api.scryfall.com/bulk-data')
allcards_url = response.json()['data'][3]['download_uri']
response = requests.get(allcards_url)
cards = response.json()
card_df = pd.DataFrame(cards)

# filter for only english language cards
card_df = card_df[card_df['lang']=='en'].copy()

# filter for only paper playable cards
card_df = card_df[pd.DataFrame(card_df.games.tolist()).isin(['paper']).any(1).values].copy()

cards_df = card_df.copy()

# split out prices and legalities columns
df2 = pd.io.json.json_normalize(card_df['legalities'])
df3 = pd.io.json.json_normalize(card_df['prices'])
df2 = df2[['commander']].copy()
df3 = df3[['usd']].copy()
card_df = pd.concat([card_df.reset_index(),df2,df3],axis=1).drop(['legalities','prices'],axis=1)

# keep only commander legal cards
card_df = card_df[card_df['commander']=='legal'].copy()

# sort columns by name and price, and keep only lowest price for every name
card_df['usd'] = card_df['usd'].astype(float)
card_df = card_df.sort_values(['name','usd'])
card_df = card_df.drop_duplicates('name')
card_df = card_df[['name','set_name','mana_cost','cmc','color_identity','edhrec_rank','usd']].copy()

# create a new column based on 2dh legality
def less_than_2(price):
    if price > 2:
        return 'illegal'
    if price <= 2:
        return 'legal'

# turn the result into a spreadsheet    
card_df['2dh_legal'] = card_df.apply(lambda row: less_than_2(row['usd']),axis=1)
card_df.sort_values(['edhrec_rank'],inplace=True)
card_df.reset_index(inplace=True)

# save the result as a new dated spreadsheet (remove the # in the next line to activate)    
#card_df.to_excel(f'2dh-{date.today()}.xlsx',index=False)

card_df.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,index,name,set_name,mana_cost,cmc,color_identity,edhrec_rank,usd,2dh_legal
0,12481,Sol Ring,Neon Dynasty Commander,{1},1.0,[],1.0,1.27,legal
1,21999,Command Tower,Commander Legends,,0.0,[],2.0,0.20,legal
2,26882,Arcane Signet,Neon Dynasty Commander,{2},2.0,[],3.0,0.87,legal
3,47812,Exotic Orchard,Neon Dynasty Commander,,0.0,[],9.0,0.23,legal
4,39752,Lightning Greaves,Archenemy,{2},2.0,[],10.0,7.39,illegal


In [6]:
# compare two spreadsheets to return lists of newly legalized cards and newly illegal cards
old_df = pd.read_excel('2dh-2022-02-10.xlsx').set_index('name')
new_df = pd.read_excel('2dh-2022-02-23.xlsx').set_index('name')
new_legal = new_df[new_df['2dh_legal']=='legal']
old_legal = old_df[old_df['2dh_legal']=='legal']
print('Newly Legal:')
display(new_legal[new_legal.index.isin(old_legal.index)==False].head(50).drop('index',axis=1))
print('Newly Illegal:')
display(new_df.loc[old_legal[old_legal.index.isin(new_legal.index)==False].index].head(50).drop('index',axis=1))

Newly Legal:


,set_name,mana_cost,cmc,color_identity,edhrec_rank,usd,2dh_legal
name,,,,,,,
Fellwar Stone,Neon Dynasty Commander,{2},2.0,[],27.0,1.29,legal
Rhythm of the Wild,Neon Dynasty Commander,{1}{R}{G},3.0,"['G', 'R']",215.0,1.26,legal
Gitaxian Probe,New Phyrexia,{U/P},1.0,['U'],245.0,1.99,legal
Bedevil,Forgotten Realms Commander,{B}{B}{R},3.0,"['B', 'R']",435.0,1.96,legal
Miscast,Core Set 2021,{U},1.0,['U'],743.0,1.82,legal
"Krenko, Tin Street Kingpin",Neon Dynasty Commander,{2}{R},3.0,['R'],892.0,0.99,legal
"Emry, Lurker of the Loch",Neon Dynasty Commander,{2}{U},3.0,['U'],985.0,1.09,legal
"Sai, Master Thopterist",Neon Dynasty Commander,{2}{U},3.0,['U'],1000.0,1.79,legal
Reflections of Littjara,Kaldheim,{4}{U},5.0,['U'],1021.0,1.73,legal


Newly Illegal


,set_name,mana_cost,cmc,color_identity,edhrec_rank,usd,2dh_legal
name,,,,,,,
Talisman of Hierarchy,Modern Horizons,{2},2.0,"['B', 'W']",221.0,3.06,illegal
Wilderness Reclamation,Ravnica Allegiance,{3}{G},4.0,['G'],440.0,2.04,illegal
Blue Sun's Zenith,Masters 25,{X}{U}{U}{U},3.0,['U'],469.0,2.12,illegal
Well of Lost Dreams,Commander 2021,{4},4.0,[],800.0,2.06,illegal
"Vivien, Champion of the Wilds",War of the Spark Promos,{2}{G},3.0,['G'],904.0,2.33,illegal
Setessan Champion,Theros Beyond Death Promos,{2}{G},3.0,['G'],909.0,2.33,illegal
"Tetsuko Umezawa, Fugitive",Dominaria,{1}{U},2.0,['U'],1059.0,3.00,illegal
Herald of the Pantheon,Magic Origins,{1}{G},2.0,['G'],1320.0,2.03,illegal
Necroblossom Snarl,Strixhaven: School of Mages,NaN,0.0,"['B', 'G']",1405.0,2.21,illegal
